In [1]:
# https://www.kaggle.com/virajbagal/fast-ai-mixup-cutmix-augmix-and-gridmask-visuals
# https://www.kaggle.com/ipythonx/keras-grapheme-gridmask-augmix-ensemble

In [194]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm.auto import tqdm
from glob import glob
import time, gc
import cv2

from tensorflow import keras
import matplotlib.image as mpimg
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.models import clone_model
from keras.layers import Dense,Conv2D,Flatten,MaxPool2D,Dropout,BatchNormalization, Input
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import PIL.Image as Image, PIL.ImageDraw as ImageDraw, PIL.ImageFont as ImageFont
from matplotlib import pyplot as plt
import seaborn as sns
from keras.utils import Sequence
from pathlib import Path
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

import albumentations
from albumentations.core.transforms_interface import DualTransform, ImageOnlyTransform
from albumentations.augmentations import functional as F

In [184]:
train_df_ = pd.read_csv('bengaliai-cv19/train.csv')


In [185]:
print(class_map_df.head())
class_map_df['component_type'].value_counts()

  component_type  label component
0  grapheme_root      0         ং
1  grapheme_root      1         ঃ
2  grapheme_root      2         অ
3  grapheme_root      3         আ
4  grapheme_root      4         ই


grapheme_root          168
vowel_diacritic         11
consonant_diacritic      7
Name: component_type, dtype: int64

In [291]:
IMG_SIZE=64
SEED = 999
N_CHANNELS = 1
im_path = 'bengaliai-cv19/'
stats = (0.0692, 0.2051)
datadir = Path('bengaliai-cv19/')
featherdir = Path('bengaliai-cv19/bengaliaicv19feather')
outdir = Path('.')

HEIGHT = 137
WIDTH = 236


In [186]:
# def prepare_image(datadir, featherdir, data_type='train',
#                   submission=False, indices=[0, 1, 2, 3]):
#     assert data_type in ['train', 'test']
#     if submission:
#         image_df_list = [pd.read_parquet(datadir / f'{data_type}_image_data_{i}.parquet')
#                          for i in indices]
#     else:
#         image_df_list = [pd.read_feather(featherdir / f'{data_type}_image_data_{i}.feather')
#                          for i in indices]

#     print('image_df_list', len(image_df_list))
#     HEIGHT = 137
#     WIDTH = 236
#     images = [df.iloc[:, 1:].values.reshape(-1, HEIGHT, WIDTH) for df in image_df_list]
#     del image_df_list
#     gc.collect()
#     images = np.concatenate(images, axis=0)
#     return images

# # train = pd.read_csv(datadir/'train.csv')
# # train_labels = train[['grapheme_root', 'vowel_diacritic', 'consonant_diacritic']].values
# indices = [0, 1, 2, 3]
# train_images = prepare_image(
#     datadir, featherdir, data_type='train', submission=False, indices=indices)


In [209]:
tmp = pd.merge(pd.read_parquet(f'bengaliai-cv19/train_image_data_0.parquet'), train_df_, on='image_id').drop(['image_id'], axis=1)
    

In [214]:
tmp#.shape

,0,1,2,3,4,5,6,7,8,9,...,32326,32327,32328,32329,32330,32331,grapheme_root,vowel_diacritic,consonant_diacritic,grapheme
0,254,253,252,253,251,252,253,251,251,253,...,253,253,253,253,253,251,15,9,5,ক্ট্রো
1,251,244,238,245,248,246,246,247,251,252,...,255,255,255,255,255,254,159,0,0,হ
2,251,250,249,250,249,245,247,252,252,252,...,253,253,253,253,251,249,22,3,5,খ্রী
3,247,247,249,253,253,252,251,251,250,250,...,254,253,253,252,251,252,53,2,2,র্টি
4,249,248,246,246,248,244,242,242,229,225,...,255,255,255,255,255,255,71,9,5,থ্রো
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50205,247,250,250,250,252,250,250,250,251,250,...,249,248,250,247,243,243,103,9,5,ফ্রো
50206,250,251,250,244,245,243,246,248,246,247,...,255,255,255,255,255,253,42,2,1,ছিঁ
50207,246,245,246,246,245,245,247,247,245,246,...,245,248,249,246,243,245,65,9,0,ত্তো
50208,246,242,240,244,247,242,239,241,245,245,...,247,246,246,248,248,247,24,0,0,গ্ধ


In [213]:
train_df_.shape

(200840, 5)

In [392]:
class GraphemeGenerator(Sequence):
    def __init__(self, data, batch_size, IMG_SIZE, shuffle=True, transform=None):
        self._data = data
        self._label_1 = pd.get_dummies(self._data['grapheme_root'], 
                                       columns = ['grapheme_root'])
        self._label_2 = pd.get_dummies(self._data['vowel_diacritic'], 
                                       columns = ['vowel_diacritic'])
        self._label_3 = pd.get_dummies(self._data['consonant_diacritic'], 
                                       columns = ['consonant_diacritic'])
        self._list_idx = data.index.values
        
        self._data = self._data.drop(['grapheme_root', 'vowel_diacritic', 'consonant_diacritic','grapheme'], axis=1)    
        
        self._batch_size = batch_size
        self._dim = (IMG_SIZE,IMG_SIZE)
        self._img_size = IMG_SIZE
        self._shuffle = shuffle
        self.transform = transform
        self.on_epoch_end()
        
    def __len__(self):
        return int(np.floor(len(self._data)/self._batch_size))
    
    def __getitem__(self, index):
        batch_idx = self._indices[index*self._batch_size:(index+1)*self._batch_size]
        _idx = [self._list_idx[k] for k in batch_idx]

        Data     = np.empty((self._batch_size, *self._dim, 1))
        Target_1 = np.empty((self._batch_size, 168), dtype = int)
        Target_2 = np.empty((self._batch_size, 11 ), dtype = int)
        Target_3 = np.empty((self._batch_size,  7 ), dtype = int)
        
        for i, k in enumerate(_idx):
            # load the image file using cv2

            image=self._data.loc[k].values.reshape(137,236)
#             image = cv2.imread(im_path + self._data['image_id'][k] + '.png')
            
            # Cropping 부분이 속도를 에폭당 40초를 늘린다.
            image = crop_char_image(image)
            image = cv2.resize(image,  self._dim) 
            # 에폭당 30초 더걸림
#             image =  crop_resize(image)
            image = image.reshape(self._img_size,self._img_size,1)
        
            if self.transform is not None:
                if np.random.rand() > 0.7:
                    # albumentation : grid mask
                    res = self.transform(image=image)
                    image = res['image']
                    # Affine Image
                    image = affine_image(image)
                    
                    
#                 else:
                    # augmix augmentation
#                     image = augment_and_mix(image)
            
            # scaling 
            image = (image.astype(np.float32)/255.0 - stats[0])/stats[1]
            
            # gray scaling 
#             gray = lambda rgb : np.dot(rgb[... , :3] , [0.299 , 0.587, 0.114]) 
#             image = gray(image)  
            
#             # expand the axises 
#             image = image[:, :, np.newaxis]

            
            Data[i,:, :, :] =  image
        
            Target_1[i,:] = self._label_1.loc[k, :].values
            Target_2[i,:] = self._label_2.loc[k, :].values
            Target_3[i,:] = self._label_3.loc[k, :].values
            
        return Data, [Target_1, Target_2, Target_3]
    
    
    def on_epoch_end(self):
        self._indices = np.arange(len(self._list_idx))
        if self._shuffle:
            np.random.shuffle(self._indices)

In [388]:
# 에폭당 40초 이상 더 걸린다.
def crop_char_image(image, threshold=5./255.):
    assert image.ndim == 2
    is_black = image > threshold

    is_black_vertical = np.sum(is_black, axis=0) > 0
    is_black_horizontal = np.sum(is_black, axis=1) > 0
    left = np.argmax(is_black_horizontal)
    right = np.argmax(is_black_horizontal[::-1])
    top = np.argmax(is_black_vertical)
    bottom = np.argmax(is_black_vertical[::-1])
    height, width = image.shape
    cropped_image = image[left:height - right, top:width - bottom]
    return cropped_image

In [389]:
# 에폭당 30초 더걸림
# helper function
def bbox(img):
    rows = np.any(img, axis=1)
    cols = np.any(img, axis=0)
    rmin, rmax = np.where(rows)[0][[0, -1]]
    cmin, cmax = np.where(cols)[0][[0, -1]]
    return rmin, rmax, cmin, cmax

def crop_resize(img0, size=IMG_SIZE, pad=16):
    #crop a box around pixels large than the threshold 
    #some images contain line at the sides
    ymin,ymax,xmin,xmax = bbox(img0[5:-5,5:-5] > 80)
    #cropping may cut too much, so we need to add it back
    xmin = xmin - 13 if (xmin > 13) else 0
    ymin = ymin - 10 if (ymin > 10) else 0
    xmax = xmax + 13 if (xmax < WIDTH - 13) else WIDTH
    ymax = ymax + 10 if (ymax < HEIGHT - 10) else HEIGHT
    img = img0[ymin:ymax,xmin:xmax]
    #remove lo intensity pixels as noise
    img[img < 28] = 0
    lx, ly = xmax-xmin,ymax-ymin
    l = max(lx,ly) + pad
    #make sure that the aspect ratio is kept in rescaling
    img = np.pad(img, [((l-ly)//2,), ((l-lx)//2,)], mode='constant')
    
    return cv2.resize(img,(size,size))

In [368]:
# augmix : https://github.com/google-research/augmix

from PIL import Image
from PIL import ImageOps
import numpy as np

def int_parameter(level, maxval):
    """Helper function to scale `val` between 0 and maxval .
    Args:
    level: Level of the operation that will be between [0, `PARAMETER_MAX`].
    maxval: Maximum value that the operation can have. This will be scaled to
      level/PARAMETER_MAX.
    Returns:
    An int that results from scaling `maxval` according to `level`.
    """
    return int(level * maxval / 10)


def float_parameter(level, maxval):
    """Helper function to scale `val` between 0 and maxval.
    Args:
    level: Level of the operation that will be between [0, `PARAMETER_MAX`].
    maxval: Maximum value that the operation can have. This will be scaled to
      level/PARAMETER_MAX.
    Returns:
    A float that results from scaling `maxval` according to `level`.
    """
    return float(level) * maxval / 10.

def sample_level(n):
    return np.random.uniform(low=0.1, high=n)

def autocontrast(pil_img, _):
    return ImageOps.autocontrast(pil_img)

def equalize(pil_img, _):
    return ImageOps.equalize(pil_img)

def posterize(pil_img, level):
    level = int_parameter(sample_level(level), 4)
    return ImageOps.posterize(pil_img, 4 - level)

def rotate(pil_img, level):
    degrees = int_parameter(sample_level(level), 30)
    if np.random.uniform() > 0.5:
        degrees = -degrees
    return pil_img.rotate(degrees, resample=Image.BILINEAR)

def solarize(pil_img, level):
    level = int_parameter(sample_level(level), 256)
    return ImageOps.solarize(pil_img, 256 - level)

def shear_x(pil_img, level):
    level = float_parameter(sample_level(level), 0.3)
    if np.random.uniform() > 0.5:
        level = -level
    return pil_img.transform((IMG_SIZE, IMG_SIZE),
                           Image.AFFINE, (1, level, 0, 0, 1, 0),
                           resample=Image.BILINEAR)

def shear_y(pil_img, level):
    level = float_parameter(sample_level(level), 0.3)
    if np.random.uniform() > 0.5:
        level = -level
    return pil_img.transform((IMG_SIZE, IMG_SIZE),
                           Image.AFFINE, (1, 0, 0, level, 1, 0),
                           resample=Image.BILINEAR)

def translate_x(pil_img, level):
    level = int_parameter(sample_level(level), IMG_SIZE / 3)
    if np.random.random() > 0.5:
        level = -level
    return pil_img.transform((IMG_SIZE, IMG_SIZE),
                           Image.AFFINE, (1, 0, level, 0, 1, 0),
                           resample=Image.BILINEAR)


def translate_y(pil_img, level):
    level = int_parameter(sample_level(level), IMG_SIZE / 3)
    if np.random.random() > 0.5:
        level = -level
    return pil_img.transform((IMG_SIZE, IMG_SIZE),
                           Image.AFFINE, (1, 0, 0, 0, 1, level),
                           resample=Image.BILINEAR)

augmentations = [
    autocontrast, equalize, posterize, rotate, solarize, shear_x, shear_y,
    translate_x, translate_y
]

# taken from https://www.kaggle.com/iafoss/image-preprocessing-128x128
MEAN = [ 0.06922848809290576,  0.06922848809290576,  0.06922848809290576]
STD = [ 0.20515700083327537,  0.20515700083327537,  0.20515700083327537]

def normalize(image):
    """Normalize input image channel-wise to zero mean and unit variance."""
    image = image.transpose(2, 0, 1)  # Switch to channel-first
    mean, std = np.array(MEAN), np.array(STD)
    image = (image - mean[:, None, None]) / std[:, None, None]
    return image.transpose(1, 2, 0)
#     return image
    


def apply_op(image, op, severity):
    image = np.clip(image * 255., 0, 255).astype(np.uint8)
    print(image.shape)
    pil_img = Image.fromarray(image)  # Convert to PIL.Image
    pil_img = op(pil_img, severity)
    return np.asarray(pil_img) / 255.


def augment_and_mix(image, severity=1, width=3, depth=1, alpha=1.):
    """Perform AugMix augmentations and compute mixture.
    Args:
    image: Raw input image as float32 np.ndarray of shape (h, w, c)
    severity: Severity of underlying augmentation operators (between 1 to 10).
    width: Width of augmentation chain
    depth: Depth of augmentation chain. -1 enables stochastic depth uniformly
      from [1, 3]
    alpha: Probability coefficient for Beta and Dirichlet distributions.
    Returns:
    mixed: Augmented and mixed image.
  """
    ws = np.float32(
      np.random.dirichlet([alpha] * width))
    m = np.float32(np.random.beta(alpha, alpha))

    mix = np.zeros_like(image)
    for i in range(width):
        image_aug = image.copy()
        depth = depth if depth > 0 else np.random.randint(1, 4)
        
        for _ in range(depth):
            op = np.random.choice(augmentations)
            image_aug = apply_op(image_aug, op, severity)
        mix = np.add(mix, ws[i] * normalize(image_aug), out=mix, 
                     casting="unsafe")

    mixed = (1 - m) * normalize(image) + m * mix
    return mixed

In [369]:
class GridMask(DualTransform):
    """GridMask augmentation for image classification and object detection.

    Args:
        num_grid (int): number of grid in a row or column.
        fill_value (int, float, lisf of int, list of float): value for dropped pixels.
        rotate ((int, int) or int): range from which a random angle is picked. If rotate is a single int
            an angle is picked from (-rotate, rotate). Default: (-90, 90)
        mode (int):
            0 - cropout a quarter of the square of each grid (left top)
            1 - reserve a quarter of the square of each grid (left top)
            2 - cropout 2 quarter of the square of each grid (left top & right bottom)

    Targets:
        image, mask

    Image types:
        uint8, float32

    Reference:
    |  https://arxiv.org/abs/2001.04086
    |  https://github.com/akuxcw/GridMask
    """

    def __init__(self, num_grid=3, fill_value=0, rotate=0, mode=0, always_apply=False, p=0.5):
        super(GridMask, self).__init__(always_apply, p)
        if isinstance(num_grid, int):
            num_grid = (num_grid, num_grid)
        if isinstance(rotate, int):
            rotate = (-rotate, rotate)
        self.num_grid = num_grid
        self.fill_value = fill_value
        self.rotate = rotate
        self.mode = mode
        self.masks = None
        self.rand_h_max = []
        self.rand_w_max = []

    def init_masks(self, height, width):
        if self.masks is None:
            self.masks = []
            n_masks = self.num_grid[1] - self.num_grid[0] + 1
            for n, n_g in enumerate(range(self.num_grid[0], self.num_grid[1] + 1, 1)):
                grid_h = height / n_g
                grid_w = width / n_g
                this_mask = np.ones((int((n_g + 1) * grid_h), int((n_g + 1) * grid_w))).astype(np.uint8)
                for i in range(n_g + 1):
                    for j in range(n_g + 1):
                        this_mask[
                             int(i * grid_h) : int(i * grid_h + grid_h / 2),
                             int(j * grid_w) : int(j * grid_w + grid_w / 2)
                        ] = self.fill_value
                        if self.mode == 2:
                            this_mask[
                                 int(i * grid_h + grid_h / 2) : int(i * grid_h + grid_h),
                                 int(j * grid_w + grid_w / 2) : int(j * grid_w + grid_w)
                            ] = self.fill_value
                
                if self.mode == 1:
                    this_mask = 1 - this_mask

                self.masks.append(this_mask)
                self.rand_h_max.append(grid_h)
                self.rand_w_max.append(grid_w)

    def apply(self, image, mask, rand_h, rand_w, angle, **params):
        h, w = image.shape[:2]
        mask = F.rotate(mask, angle) if self.rotate[1] > 0 else mask
        mask = mask[:,:,np.newaxis] if image.ndim == 3 else mask
        image *= mask[rand_h:rand_h+h, rand_w:rand_w+w].astype(image.dtype)
        return image

    def get_params_dependent_on_targets(self, params):
        img = params['image']
        height, width = img.shape[:2]
        self.init_masks(height, width)

        mid = np.random.randint(len(self.masks))
        mask = self.masks[mid]
        rand_h = np.random.randint(self.rand_h_max[mid])
        rand_w = np.random.randint(self.rand_w_max[mid])
        angle = np.random.randint(self.rotate[0], self.rotate[1]) if self.rotate[1] > 0 else 0

        return {'mask': mask, 'rand_h': rand_h, 'rand_w': rand_w, 'angle': angle}

    @property
    def targets_as_params(self):
        return ['image']

    def get_transform_init_args_names(self):
        return ('num_grid', 'fill_value', 'rotate', 'mode')

In [370]:
# 이게 느리다.

"""
From https://www.kaggle.com/corochann/deep-learning-cnn-with-chainer-lb-0-99700
"""
import cv2
from skimage.transform import AffineTransform, warp
import numpy as np


def affine_image(img):
    """

    Args:
        img: (h, w) or (1, h, w)

    Returns:
        img: (h, w)
    """
    # ch, h, w = img.shape
    # img = img / 255.
    if img.ndim == 3:
        img = img[0]

    # --- scale ---
    min_scale = 0.8
    max_scale = 1.2
    sx = np.random.uniform(min_scale, max_scale)
    sy = np.random.uniform(min_scale, max_scale)

    # --- rotation ---
    max_rot_angle = 7
    rot_angle = np.random.uniform(-max_rot_angle, max_rot_angle) * np.pi / 180.

    # --- shear ---
    max_shear_angle = 10
    shear_angle = np.random.uniform(-max_shear_angle, max_shear_angle) * np.pi / 180.

    # --- translation ---
    max_translation = 4
    tx = np.random.randint(-max_translation, max_translation)
    ty = np.random.randint(-max_translation, max_translation)

    tform = AffineTransform(scale=(sx, sy), rotation=rot_angle, shear=shear_angle,
                            translation=(tx, ty))
    transformed_image = warp(img, tform)
    assert transformed_image.ndim == 2
    return transformed_image

## Data Generation

# Model

In [361]:
from keras_applications.resnext import ResNeXt101
from keras.applications.resnet50 import ResNet50
from keras.applications.densenet import DenseNet121
from keras_efficientnets import EfficientNetB3
from keras import backend as K
from keras import layers, models, optimizers, utils, backend,regularizers
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Activation, Conv2D, GlobalAveragePooling2D
from keras import layers, models, optimizers, utils, backend,regularizers
from keras.layers import LeakyReLU
def get_model(model_name='EfficientNetB3'):
#     inputs = Input(shape = (IMG_SIZE, IMG_SIZE, 1))
    pretrained_model = EfficientNetB3(weights="imagenet", include_top=False, input_shape=(64,64,3))
#     resNet_model = DenseNet121(include_top= False, input_shape = (128,128,3)
#                             , backend =backend, layers=layers, models = models,
#                              utils = utils
#                             )
    
    
    inputs = Input(shape = (IMG_SIZE, IMG_SIZE, 1))
    model = Conv2D(3, (3, 3), padding='same')(inputs)
    model = pretrained_model(model)


#     model = resNet_model.layers[-1].output
    model = GlobalAveragePooling2D()(model)
    model = Dropout(rate=0.3)(model)
    
#     model = Flatten()(model)
    model = Dense(1024, activation = "relu")(model)
    model = Dropout(rate=0.3)(model)
    dense = Dense(512, activation = "relu")(model)
    
    head_root = Dense(168, activation = 'softmax',name = 'root')(model)
    head_vowel = Dense(11, activation = 'softmax', name ='vowel')(model)
    head_consonant = Dense(7, activation = 'softmax', name = 'consonant')(model)

    model = Model(inputs=inputs, outputs=[head_root, head_vowel, head_consonant])
    
    model.summary()
    adam = optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-8)
   
    model.compile( optimizer = adam, 
                 loss = {'root' : 'categorical_crossentropy', 
                        'vowel' : 'categorical_crossentropy', 
                        'consonant': 'categorical_crossentropy'},
               metrics = {'root' : 'accuracy', 
                        'vowel' : 'accuracy', 
                        'consonant': 'accuracy'}
                 )
#     model.compile(optimizer = adam,loss = 'categorical_crossentropy', metrics=['accuracy'])
    # compile 할때 넣어줘야지 아래에서 early stopping 할때 사용 가능하다
    return model

In [362]:
model = get_model()

Model: "model_15"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_16 (InputLayer)           (None, 64, 64, 1)    0                                            
__________________________________________________________________________________________________
conv2d_736 (Conv2D)             (None, 64, 64, 3)    30          input_16[0][0]                   
__________________________________________________________________________________________________
model_14 (Model)                (None, 2, 2, 1536)   10783528    conv2d_736[0][0]                 
__________________________________________________________________________________________________
global_average_pooling2d_8 (Glo (None, 1536)         0           model_14[1][0]                   
___________________________________________________________________________________________

# Modeling

In [363]:
# Set a learning rate annealer. Learning rate will be half after 3 epochs if accuracy is not increased
from keras.callbacks import  EarlyStopping
# root_accuracy: 0.7609 - vowel_accuracy: 0.9383 - consonant_accuracy: 0.9436 
learning_rate_reduction_root = ReduceLROnPlateau(monitor='root_accuracy', 
                                            patience=3, 
                                            verbose=1,
                                            factor=0.5, 
                                            min_lr=0.00001)
learning_rate_reduction_vowel = ReduceLROnPlateau(monitor='vowel_accuracy', 
                                            patience=3, 
                                            verbose=1,
                                            factor=0.5, 
                                            min_lr=0.00001)
learning_rate_reduction_consonant = ReduceLROnPlateau(monitor='consonant_accuracy', 
                                            patience=3, 
                                            verbose=1,
                                            factor=0.5, 
                                            min_lr=0.00001)
# Callback : Early Stop
earlyStop = EarlyStopping(monitor='val_loss',
                          mode = 'auto',
                          patience = 10,
                          min_delta = 0,
                          verbose = 1)


In [344]:
batch_size = 300
epochs = 100
HEIGHT = 137
WIDTH = 236

In [267]:
train_df = pd.merge(pd.read_parquet(f'bengaliai-cv19/train_image_data_0.parquet'), train_df_, on='image_id').drop(['image_id'], axis=1)


In [393]:
from albumentations import (
    Compose, HorizontalFlip, CLAHE, HueSaturationValue,
    RandomBrightness, RandomContrast, RandomGamma,
    ToFloat, ShiftScaleRotate
)
# grid mask augmentation
transforms_train = albumentations.Compose([
#     GridMask(num_grid=3, rotate=15, p=1),
    ShiftScaleRotate(
        shift_limit=0.0625, scale_limit=0.1, 
        rotate_limit=15, border_mode=cv2.BORDER_REFLECT_101, p=0.8), 
])



train_, valid_ = train_test_split(train_df, test_size = 0.10, random_state = SEED,
                                            stratify = train_df[['grapheme_root', 
                                                              'vowel_diacritic', 
                                                              'consonant_diacritic']])
# train_, valid_ = train_df.iloc[train_labels.index] , train_df.iloc[valid_labels.index]

# training generator
train_generator = GraphemeGenerator(train_, batch_size, IMG_SIZE, 
                                shuffle = True, transform=transforms_train)

# validation generator: no shuffle , not augmentation
val_generator = GraphemeGenerator(valid_, batch_size, IMG_SIZE, 
                              shuffle = False)


In [ ]:



# checkpoint = ModelCheckpoint('E0Train_B0.h5', 
#                                  monitor = 'val_loss', 
#                                  verbose = 0, save_best_only=True, 
#                                  mode = 'min',
#                                  save_weights_only = True)

callbacks = [learning_rate_reduction_root, learning_rate_reduction_vowel, learning_rate_reduction_consonant,earlyStop]
history = model.fit_generator(
    train_generator,
    steps_per_epoch=int(train_.shape[0]/batch_size), 
    validation_data=val_generator,
    validation_steps = int(valid_.shape[0]/batch_size),
    epochs=epochs,
    callbacks=callbacks
)

Epoch 1/100
 98/225 [============>.................] - ETA: 56s - loss: 3.5675 - root_loss: 2.3147 - vowel_loss: 0.7708 - consonant_loss: 0.4820 - root_accuracy: 0.4427 - vowel_accuracy: 0.7156 - consonant_accuracy: 0.8441

In [270]:
model.save('dense_model7_64_early.h5')

,0,1,2,3,4,5,6,7,8,9,...,32326,32327,32328,32329,32330,32331,grapheme_root,vowel_diacritic,consonant_diacritic,grapheme
0,254,253,252,253,251,252,253,251,251,253,...,253,253,253,253,253,251,15,9,5,ক্ট্রো
1,251,244,238,245,248,246,246,247,251,252,...,255,255,255,255,255,254,159,0,0,হ
2,251,250,249,250,249,245,247,252,252,252,...,253,253,253,253,251,249,22,3,5,খ্রী
3,247,247,249,253,253,252,251,251,250,250,...,254,253,253,252,251,252,53,2,2,র্টি
4,249,248,246,246,248,244,242,242,229,225,...,255,255,255,255,255,255,71,9,5,থ্রো
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50205,247,250,250,250,252,250,250,250,251,250,...,249,248,250,247,243,243,103,9,5,ফ্রো
50206,250,251,250,244,245,243,246,248,246,247,...,255,255,255,255,255,253,42,2,1,ছিঁ
50207,246,245,246,246,245,245,247,247,245,246,...,245,248,249,246,243,245,65,9,0,ত্তো
50208,246,242,240,244,247,242,239,241,245,245,...,247,246,246,248,248,247,24,0,0,গ্ধ


In [286]:
tmp = train_df.drop(['grapheme_root', 'vowel_diacritic', 'consonant_diacritic','grapheme'], axis=1).loc[0]
# print(tmp)
# tmp =tmp.
image=tmp.values.reshape(137,236)
#             image = cv2.imread(im_path + self._data['image_id'][k] + '.png')
image = cv2.resize(image, (64,64)) 
print(image.shape)
image = image.reshape((IMG_SIZE,IMG_SIZE),N_CHANNELS)
print(image.shape)

(64, 64)


TypeError: 'tuple' object cannot be interpreted as an integer

In [285]:
image

array([[[253],
        [252],
        [253],
        ...,
        [254],
        [254],
        [253]],

       [[255],
        [255],
        [255],
        ...,
        [255],
        [255],
        [252]],

       [[255],
        [255],
        [255],
        ...,
        [255],
        [255],
        [252]],

       ...,

       [[254],
        [254],
        [252],
        ...,
        [254],
        [254],
        [254]],

       [[253],
        [253],
        [252],
        ...,
        [254],
        [253],
        [254]],

       [[254],
        [254],
        [253],
        ...,
        [253],
        [253],
        [253]]], dtype=uint8)